# Appendix 0: Different input types for Hamiltonians

Author: Gediminas Kiršanskas

In this Appendix it is shown how Hamiltonian can be specified using different types (numpy array, list, dictionary). We use single orbital example from *tutorial.ipynb* notebook.

In [1]:
# Prerequisites
from __future__ import division, print_function
import numpy as np
import qmeq

### Single orbital example
Let us consider an example of a quantum dot containing one spinful orbital and on-site charging energy $U$ coupled to source ($L$) and drain ($R$) leads:

$H_{\mathrm{one}}=\sum_{\substack{\ell=L,R \\ \sigma=\uparrow,\downarrow}}\varepsilon_{\ell\sigma k}^{\phantom{\dagger}}c_{\ell\sigma k}^{\dagger}c_{\ell\sigma k}^{\phantom{\dagger}}
+\sum_{\ell\sigma k}\left(t_{\ell\sigma}c_{\ell\sigma k}^{\dagger}d_{\sigma}+\mathrm{h.c.}\right)
+\sum_{\sigma}\varepsilon_{\sigma}d_{\sigma}^{\dagger}d_{\sigma}
+(\Omega d_{\uparrow}^{\dagger}d_{\downarrow}+\mathrm{h.c.})
+Ud_{\uparrow}^{\dagger}d_{\downarrow}^{\dagger}d_{\downarrow}^{\phantom{\dagger}}d_{\uparrow}^{\phantom{\dagger}}$    

with $\varepsilon_{\uparrow}=V_{g}+\frac{B}{2}$, $\varepsilon_{\downarrow}=V_{g}\frac{B}{2}$, where $V_{g}$ is the gate voltage and $B$ is the magnetic field (representing anamolous Zeeman splitting of spinful orbital). We will choose such values for parameters:

In [2]:
# Quantum dot parameters
vgate = 1.0
bfield = 0.5
omega = 2.0
U = 20.0
# Lead parameters
vbias = 0.5
temp = 1.0
dband = 40.0
# Tunneling amplitudes
gam = 0.5
t0 = np.sqrt(gam/(2*np.pi))

In $H_{\mathrm{one}}$ we have two single particle states $\lvert\uparrow\rangle$ and $\lvert\downarrow\rangle$. The single particle Hamiltonian $H_{\mathrm{single}}$ can be specified using numpy array, a list, or a dictionary:

In [3]:
nsingle = 2

# numpy array             up                 down 
hsingle = np.array([[vgate+bfield/2,             omega],  # up
                    [omega.conjugate(), vgate-bfield/2]]) # down

# list, 0 is up, 1 is down
hsingle = [[0, 0, vgate+bfield/2], 
           [1, 1, vgate-bfield/2],
           [0, 1, omega]]

# dictionary, 0 is up, 1 is down
hsingle = {(0, 0): vgate+bfield/2,
           (1, 1): vgate-bfield/2,
           (0, 1): omega}

**Comment 1.** We note that for a list or a dictionary it is enough to specify one element like $\Omega d_{\uparrow}^{\dagger}d_{\downarrow}^{\phantom{\dagger}}$, because the other one $\Omega^{*} d_{\downarrow}^{\dagger}d_{\uparrow}^{\phantom{\dagger}}$ is determined by complex conjugation and is included automatically. Also if element like (1,0) is given then it will be added to the Hamiltonian. So specifying {(0, 1):omega, (1,0):omega.conjugate()} will simply double count $\Omega$.

**Comment 2.** QmeQ will convert any numpy array or a list to a dictionary for internal usage. Also all elements in numpy array equal to zero are not included in the Hamiltonian dictionary.

The Coulomb interaction Hamiltonian $H_{\mathrm{Coulomb}}$ is specified using either a list or a dictionary:

In [4]:
coulomb = [[0,1,1,0,U]]
coulomb = {(0,1,1,0):U}

The tunneling Hamiltonian $H_{\mathrm{tunneling}}$ can be specified as:

In [5]:
# numpy array       up down  
tleads = np.array([[t0, 0],  # L, up
                   [t0, 0],  # R, up
                   [0, t0],  # L, down                   
                   [0, t0]]) # R, down

# list                # lead label, lead spin <-- level spin
tleads = [[0, 0, t0], # L, up   <-- up
          [1, 0, t0], # R, up   <-- up
          [2, 1, t0], # L, down <-- down          
          [3, 1, t0]] # R, down <-- down

# dictionary
tleads = {(0, 0):t0,
          (1, 0):t0,
          (2, 1):t0,          
          (3, 1):t0}

**Comment 3.** In dictionary tuple $(\alpha, i)$ the first label $\alpha$ denotes lead quantum numbers, the second label $i$ denotes levels of the quantum dot. We note that effectively we get four leads, i.e., $\alpha\in\{L\uparrow,R\uparrow,L\downarrow,R\downarrow\}$.

### Specify the system using numpy arrays and lists

In [6]:
nsingle = 2
nleads = 4
# numpy array             up                 down 
hsingle = np.array([[vgate+bfield/2,             omega],  # up
                    [omega.conjugate(), vgate-bfield/2]]) # down
# numpy array       up down  
tleads = np.array([[t0, 0],  # L, up
                   [t0, 0],  # R, up
                   [0, t0],  # L, down                   
                   [0, t0]]) # R, down
coulomb = [[0,1,1,0,U]]
#        L,up      R,up     L,down    R,down
mulst = [vbias/2, -vbias/2, vbias/2, -vbias/2]
tlst = [temp, temp, temp, temp]

In [7]:
system = qmeq.Builder(nsingle, hsingle, coulomb,
                      nleads, tleads, mulst, tlst, dband,
                      kerntype='Pauli')

The numpy arrays *hsingle*, *tleads* and a list *coulomb* get converted to dictionaries in **QmeQ**

In [8]:
print(system.qd.hsingle)
print(system.qd.coulomb)
print(system.leads.tleads)

{(0, 1): 2.0, (0, 0): 1.25, (1, 1): 0.75}
{(0, 1, 1, 0): 20.0}
{(1, 0): 0.28209479177387814, (0, 0): 0.28209479177387814, (3, 1): 0.28209479177387814, (2, 1): 0.28209479177387814}
